# Assignment 3
## Econ 8310 - Business Forecasting

For homework assignment 3, you will work with [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), a more fancier data set.

- You must create a custom data loader as described in the first week of neural network lectures [2 points]
    - You will NOT receive credit for this if you use the pytorch prebuilt loader for Fashion MNIST!
- You must create a working and trained neural network using only pytorch [2 points]
- You must store your weights and create an import script so that I can evaluate your model without training it [2 points]

Highest accuracy score gets some extra credit!

Submit your forked repository URL on Canvas! :) I'll be manually grading this assignment.

Some checks you can make on your own:
- Did you manually process the data or use a prebuilt loader (see above)?
- Does your script train a neural network on the assigned data?
- Did your script save your model?
- Do you have separate code to import your model for use after training?

In [5]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import urllib.request
import gzip
import numpy as np
import struct

# custom dataloader for the Fashion-MNIST dataset
class FashionMNIST(Dataset):
    def __init__(self, train_images_url, train_labels_url, test_images_url, test_labels_url, train=True):
        self.train = train
        self.unzip(train_images_url, train_labels_url, test_images_url, test_labels_url)

    # credit to Google and its AI search assistant as my source for this section on how to unzip and use the .gz files for this custom dataloader
    def unzip(self, train_images_url, train_labels_url, test_images_url, test_labels_url):
        with urllib.request.urlopen(train_images_url) as response, gzip.GzipFile(fileobj=response) as f:
            magic, num_images, rows, cols = struct.unpack(">IIII", f.read(16))
            self.train_images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows, cols)

        with urllib.request.urlopen(train_labels_url) as response, gzip.GzipFile(fileobj=response) as f:
            magic, num_labels = struct.unpack(">II", f.read(8))
            self.train_labels = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_labels)

        with urllib.request.urlopen(test_images_url) as response, gzip.GzipFile(fileobj=response) as f:
            magic, num_images, rows, cols = struct.unpack(">IIII", f.read(16))
            self.test_images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows, cols)

        with urllib.request.urlopen(test_labels_url) as response, gzip.GzipFile(fileobj=response) as f:
            magic, num_labels = struct.unpack(">II", f.read(8))
            self.test_labels = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_labels)

    # returns the length of either the train dataset or test dataset depending on tain = true or false
    def __len__(self):
        return len(self.train_labels) if self.train else len(self.test_labels)

    # prepares the train or test dataset with imagaes matching up to labels depending on tain = true or false
    def __getitem__(self, idx):
        if self.train:
            image = torch.tensor(self.train_images[idx], dtype=torch.float32).unsqueeze(0)
            label = torch.tensor(self.train_labels[idx], dtype=torch.long)
        else:
            image = torch.tensor(self.test_images[idx], dtype=torch.float32).unsqueeze(0)
            label = torch.tensor(self.test_labels[idx], dtype=torch.long)

        return image, label

# URLs for training
train_images_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/train-images-idx3-ubyte.gz'
train_labels_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/train-labels-idx1-ubyte.gz'

# URLs for testing
test_images_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/t10k-images-idx3-ubyte.gz'
test_labels_url = 'https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/t10k-labels-idx1-ubyte.gz'

# create an instances for both the training dataset and testing dataset
train_dataset = FashionMNIST(train_images_url, train_labels_url, test_images_url, test_labels_url, train=True)
test_dataset = FashionMNIST(train_images_url, train_labels_url, test_images_url, test_labels_url, train=False)

# create the dataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

# testing the dataloader by verifying number of results
# len(train_dataset)
# len(test_dataset)

# linear Neural Network Classifier Model
class FashionNet(nn.Module):
    def __init__(self):
      super(FashionNet, self).__init__()
      self.flatten = nn.Flatten()

      self.linear_relu_model = nn.Sequential(
            nn.LazyLinear(10),
        )

    # forward function
    def forward(self, x):
      x = self.flatten(x)
      output = self.linear_relu_model(x)
      return output

model = FashionNet()

# training parameters
learning_rate = 1e-6
batch_size = 64
epochs = 100

# loss function
loss_fn = nn.CrossEntropyLoss()

# optimizer with the parameters above and learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# training class to train the model
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# testing class to test our trained model
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# train and test the model
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Streaming output truncated to the last 5000 lines.
loss: 3.741199  [30784/60000]
loss: 1.948558  [31424/60000]
loss: 2.451840  [32064/60000]
loss: 2.609213  [32704/60000]
loss: 2.047429  [33344/60000]
loss: 5.116594  [33984/60000]
loss: 1.798417  [34624/60000]
loss: 2.936960  [35264/60000]
loss: 1.774134  [35904/60000]
loss: 3.347204  [36544/60000]
loss: 1.586969  [37184/60000]
loss: 2.851919  [37824/60000]
loss: 4.491022  [38464/60000]
loss: 1.491983  [39104/60000]
loss: 3.039938  [39744/60000]
loss: 4.372824  [40384/60000]
loss: 3.174446  [41024/60000]
loss: 4.670141  [41664/60000]
loss: 3.273799  [42304/60000]
loss: 3.313775  [42944/60000]
loss: 2.533962  [43584/60000]
loss: 2.170287  [44224/60000]
loss: 4.107714  [44864/60000]
loss: 2.493140  [45504/60000]
loss: 5.691240  [46144/60000]
loss: 2.563176  [46784/60000]
loss: 3.019914  [47424/60000]
loss: 3.612900  [48064/60000]
loss: 3.247992  [48704/60000]
loss: 3.010005  [49344/60000]
loss: 3.880692  [49984/60000]
loss: 1.070965  [50

In [6]:
# saves the model so we can call it back to evaluate without training
EPOCH = epochs
PATH = "model.pt"
torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, PATH)

In [7]:
# recall our model and create an instance of it
PATH = "model.pt"
model = FashionNet()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

# reload all of our data from the file
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
EPOCH = checkpoint['epoch']